In [1]:
# import torch
# from transformers import T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from private_transformers import PrivacyEngine
# import torch.nn.functional as F
# # from transformers import AdamW, get_linear_schedule_with_warmup
# # from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, get_linear_schedule_with_warmup
# from torch.optim import AdamW  
# import numpy as np
# import os
# import wandb
# from datasets import Dataset
# import pandas as pd

# import spacy
# spacy_model = spacy.load("en_core_web_sm")


# def preprocess_function(batch, tokenizer, max_input_length=512, max_output_length=80):
#     inputs = tokenizer(batch["dialogue"], max_length=max_input_length, padding="max_length", truncation=True, return_tensors='pt')
#     targets = tokenizer(batch["summary"], max_length=max_output_length, padding="max_length", truncation=True, return_tensors='pt')
    
#     print("Inputs: ",inputs )
#     print("Inputs: ",targets )
#     # Preparing the dataset to be returned
#     model_inputs = {key: inputs[key] for key in inputs}
#     model_inputs["labels"] = targets["input_ids"]
    
#     return model_inputs


# def find_latest_checkpoint(checkpoint_dir):
#     checkpoint_subdirs = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))]
#     if not checkpoint_subdirs:
#         return None
#     latest_checkpoint = max(checkpoint_subdirs, key=lambda d: int(d.split('-')[-1]))
#     return os.path.join(checkpoint_dir, latest_checkpoint)

# def compute_average_length(dataset):
#     lengths = dataset["lengths"]  # Assuming 'lengths' are stored in the dataset
#     average_length = np.mean(lengths)
#     print(f"Average token length: {average_length}")

# wandb.init(project="Summarize_DP_Dialog", entity="ivolinengong", name="dp")

# # Check if CUDA is available and set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Set seeds for reproducibility
# torch.manual_seed(42)

# # Load the tokenizer
# max_input_length = 200 #1024
# max_output_length = 100 #256 #250
# batch_size = 5
# epochs = 8
# gradient_accumulation_steps = 10

# # Load SAMSum dataset
# train_dataset = load_dataset("samsum", split="train")
# val_dataset = load_dataset("samsum", split="validation")

# # Checkpoint directories
# checkpoint_dir = f"./dp/dp_results/{wandb.run.name}"
# # checkpoint_dir = f"./results/dp_results/test/rich-serenity-9"
# os.makedirs(checkpoint_dir, exist_ok=True)

# model_tokenizer_path = checkpoint_dir
# optimizer_checkpoint_path = os.path.join(checkpoint_dir, "optimizer_and_loss.pth")

# # Load model and tokenizer, and optimizer state if available
# model_checkpoint = find_latest_checkpoint(checkpoint_dir)
# if model_checkpoint:
#     print(f"Resuming from checkpoint: {model_checkpoint}")

#     # Load model and tokenizer from the checkpoint
#     model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
#     tokenizer = T5Tokenizer.from_pretrained(model_checkpoint, model_max_length=max_input_length)

#     # Load optimizer state
#     if os.path.isfile(optimizer_checkpoint_path):
#         print("Loading optimizer state and best loss from checkpoint")
#         checkpoint = torch.load(optimizer_checkpoint_path, map_location=device)
#         optimizer = AdamW(model.parameters(), lr=5e-5)
#         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#         best_loss = checkpoint['best_loss']
# else:
#     print("Starting training from the pretrained model")
#     model = T5ForConditionalGeneration.from_pretrained("t5-large").to(device)
#     tokenizer = T5Tokenizer.from_pretrained("t5-large", model_max_length=max_input_length)
#     optimizer = AdamW(model.parameters(), lr=5e-5)
#     best_loss = float('inf')

# # Initialize tokenizer and model
# tokenizer = T5Tokenizer.from_pretrained("t5-base")
# model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)

# # Tokenize and preprocess datasets
# tokenized_train_dataset = train_dataset.map(lambda batch: preprocess_function(batch, tokenizer, max_input_length, max_output_length), batched=True)
# # tokenized_val_dataset = val_dataset.map(lambda batch: preprocess_function(batch, tokenizer, max_input_length, max_output_length), batched=True)

# print("\nTraining dataset size:", len(tokenized_train_dataset))

# # # Inspect a sample
# sample = tokenized_train_dataset[0]
# # print(f"Sample keys: {sample.keys()}")
# # print(f"Input IDs shape: {sample['input_ids'].shape}")
# # print(f"Attention mask shape: {sample['attention_mask'].shape}")
# # print(f"Labels shape: {sample['labels'].shape}")

# print(f"Sample keys: {sample.keys()}")
# print(f"Input IDs shape: {torch.tensor(sample['input_ids']).shape}")
# print(f"Attention mask shape: {torch.tensor(sample['attention_mask']).shape}")
# print(f"Labels shape: {torch.tensor(sample['labels']).shape}")


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Attach the privacy engine to the optimizer
# privacy_engine = PrivacyEngine(
#     model,
#     batch_size=batch_size,
#     sample_size=len(tokenized_train_dataset),
#     epochs=epochs,
#     max_grad_norm=1.0,
#     target_epsilon=8,
#     noise_multiplier=0.1,
#     # clipping_mode="ghost",
# )
# privacy_engine.attach(optimizer)

# # Learning rate scheduler
# total_steps = len(tokenized_train_dataset) // batch_size * epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# data_loader = DataLoader(tokenized_train_dataset, batch_size=batch_size, shuffle=True)

# # Training loop
# model.train()
# for epoch in range(epochs):
#     total_loss = 0
#     print(f"\nStarting epoch {epoch + 1}/{epochs}", flush=True)
#     for i, batch in enumerate(data_loader):
#         # input_ids = batch['input_ids'].to(device)
#         # attention_mask = batch['attention_mask'].to(device)
#         # labels = batch['labels'].to(device)

#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         # input_ids = torch.tensor(batch['input_ids']).to(device)
#         # attention_mask = torch.tensor(batch['attention_mask']).to(device)
#         # labels = torch.tensor(batch['labels']).to(device)

#         # Forward pass
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         logits = outputs.logits

#         # Compute per-example loss
#         shift_logits = logits[..., :-1, :].contiguous()
#         shift_labels = labels[..., 1:].contiguous()
#         loss = F.cross_entropy(shift_logits.permute(0, 2, 1), shift_labels, reduction="none").mean(dim=1)

#         # # Optimization step (without calling loss.backward())
#         # optimizer.step(loss=loss)

#         # optimizer.zero_grad()
#         # scheduler.step()
        
#         # Gradient accumulation
#         if (i + 1) % gradient_accumulation_steps == 0:
#             optimizer.step(loss=loss)  # Perform parameter update
#             optimizer.zero_grad()  # Reset gradients
#             scheduler.step()
#         else:
#             optimizer.virtual_step(loss=loss)  # Accumulate gradients


#         # Save the model if this is the best loss so far
#         avg_loss = torch.mean(loss).item()
#         total_loss += avg_loss
#         # Log metrics to wandb
#         # wandb.log({"Epoch": epoch + 1, "Step": i, "Loss": avg_loss})
        
#         # print(f"Epoch: {epoch + 1}, Step: {i}/{len(data_loader)}, Loss: {avg_loss}\n")

#         # Check if the current loss is the best
#         if avg_loss < best_loss:
#             best_loss = avg_loss

#             # Save model and tokenizer using save_pretrained
#             model.save_pretrained(checkpoint_dir)
#             tokenizer.save_pretrained(checkpoint_dir)

#             # Save optimizer state and best_loss separately
#             torch.save({
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'best_loss': best_loss,
#             }, optimizer_checkpoint_path)

#             print(f"New best model saved with loss: {best_loss}")

#         # # Log progress
#         if i % 10 == 0:
#             print(f"Epoch: {epoch + 1}, Step: {i}/{len(data_loader)}, Loss: {avg_loss}", flush=True)

#     # Calculate the average training loss and perplexity for the epoch
#     avg_train_loss = total_loss / len(data_loader)
#     train_perplexity = np.exp(avg_train_loss)
#     wandb.log({"epoch": epoch + 1, "train_loss": avg_train_loss, "train_perplexity": train_perplexity})

In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

from datasets import load_dataset
from torch.utils.data import DataLoader
from private_transformers import PrivacyEngine
import torch.nn.functional as F
# from transformers import AdamW, get_linear_schedule_with_warmup
# from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, get_linear_schedule_with_warmup
from torch.optim import AdamW  
import numpy as np
import os
import wandb
import spacy

# Load the SpaCy model for sentence tokenization
spacy_model = spacy.load("en_core_web_sm")


# def preprocess_function(examples):
#     # Use spacy to extract first 3 sentences as a simplification
#     def get_first_3_sentences(text):
#         doc = spacy_model(text)
#         sentences = [sent.text.strip() for sent in doc.sents][:3]
#         return " ".join(sentences)
    
#     # Modify here to use 'dialogue' field from SAMSum dataset
#     inputs = ["summarize: " + get_first_3_sentences(doc) for doc in examples["dialogue"]]
#     model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

#     # 'summary' field is used as labels in SAMSum
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples["summary"], max_length=max_output_length, padding="max_length", truncation=True)

#     model_inputs["input_ids"] = torch.tensor(model_inputs["input_ids"])
#     model_inputs["attention_mask"] = torch.tensor(model_inputs["attention_mask"])
#     model_inputs["labels"] = torch.tensor(labels["input_ids"])
#     return model_inputs

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

    # 'summary' field is used as labels in SAMSum
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_output_length, padding="max_length", truncation=True)

    model_inputs["input_ids"] = torch.tensor(model_inputs["input_ids"])
    model_inputs["attention_mask"] = torch.tensor(model_inputs["attention_mask"])
    model_inputs["labels"] = torch.tensor(labels["input_ids"])
    return model_inputs

def find_latest_checkpoint(checkpoint_dir):
    checkpoint_subdirs = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))]
    if not checkpoint_subdirs:
        return None
    latest_checkpoint = max(checkpoint_subdirs, key=lambda d: int(d.split('-')[-1]))
    return os.path.join(checkpoint_dir, latest_checkpoint)

def compute_average_length(dataset):
    lengths = dataset["lengths"]  # Assuming 'lengths' are stored in the dataset
    average_length = np.mean(lengths)
    print(f"Average token length: {average_length}")

wandb.init(project="Summarize_DP_PubMed", entity="ivolinengong")


# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds for reproducibility
torch.manual_seed(42)

# Load the tokenizer
max_input_length = 150 #1024
max_output_length = 100 #256 #250
batch_size = 5
epochs = 8
gradient_accumulation_steps = 10

# Load pubmed dataset
# Load SAMSum dataset
train_dataset = load_dataset("samsum", split="train")
val_dataset = load_dataset("samsum", split="validation")

# tokenized_datasets = dataset.map(preprocess_function, batched=True)
# train_dataset = tokenized_datasets["train")

# Checkpoint directories
# checkpoint_dir = "./results/dp_results/test/run"
checkpoint_dir = f"./results/dp_results/{wandb.run.name}"
# checkpoint_dir = f"./results/dp_results/test/rich-serenity-9"
os.makedirs(checkpoint_dir, exist_ok=True)

model_tokenizer_path = checkpoint_dir
optimizer_checkpoint_path = os.path.join(checkpoint_dir, "optimizer_and_loss.pth")

# Load model and tokenizer, and optimizer state if available
# model_checkpoint = find_latest_checkpoint(checkpoint_dir)
# if model_checkpoint:
#     print(f"Resuming from checkpoint: {model_checkpoint}")
#     # model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)
#     # tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#     model = T5ForConditionalGeneration.from_pretrained("t5-large").to(device)
#     tokenizer = T5Tokenizer.from_pretrained("t5-large", model_max_length=max_input_length)

#     if os.path.isfile(optimizer_checkpoint_path):
#         print("Loading optimizer state and best loss from checkpoint")
#         checkpoint = torch.load(optimizer_checkpoint_path, map_location=device)
#         optimizer = AdamW(model.parameters(), lr=5e-5)
#         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#         best_loss = checkpoint['best_loss']
model_checkpoint = find_latest_checkpoint(checkpoint_dir)
if model_checkpoint:
    print(f"Resuming from checkpoint: {model_checkpoint}")

    # Load model and tokenizer from the checkpoint
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint, model_max_length=max_input_length)

    # Load optimizer state
    if os.path.isfile(optimizer_checkpoint_path):
        print("Loading optimizer state and best loss from checkpoint")
        checkpoint = torch.load(optimizer_checkpoint_path, map_location=device)
        optimizer = AdamW(model.parameters(), lr=5e-5)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        best_loss = checkpoint['best_loss']
else:
    print("Starting training from the pretrained model")
    model = T5ForConditionalGeneration.from_pretrained("t5-large").to(device)
    tokenizer = T5Tokenizer.from_pretrained("t5-large", model_max_length=max_input_length)

    # model = AutoModelForSeq2SeqLM.from_pretrained("t5-large").to(device)
    # tokenizer = AutoTokenizer.from_pretrained("t5-large", model_max_length=max_input_length)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    best_loss = float('inf')

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


print("\nTraining dataset size:", len(tokenized_train_dataset))
print("Validation dataset size:", len(tokenized_val_dataset))


# # Inspect a sample
sample = tokenized_train_dataset[0]
print(f"Sample keys: {sample.keys()}")
print(f"Input IDs shape: {sample['input_ids'].shape}")
print(f"Attention mask shape: {sample['attention_mask'].shape}")
print(f"Labels shape: {sample['labels'].shape}")
# Calculate and print average token length
# compute_average_length(tokenized_train_dataset)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Attach the privacy engine to the optimizer
privacy_engine = PrivacyEngine(
    model,
    batch_size=batch_size,
    sample_size=len(tokenized_train_dataset),
    epochs=epochs,
    max_grad_norm=1.0,
    target_epsilon=3,
    noise_multiplier=0.1,
    # clipping_mode="ghost",
)
privacy_engine.attach(optimizer)

# Learning rate scheduler
total_steps = len(tokenized_train_dataset) // batch_size * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Data loader
data_loader = DataLoader(tokenized_train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(tokenized_val_dataset, batch_size=batch_size)


# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    print(f"\nStarting epoch {epoch + 1}/{epochs}", flush=True)
    for i, batch in enumerate(data_loader):
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        # Compute per-example loss
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        loss = F.cross_entropy(shift_logits.permute(0, 2, 1), shift_labels, reduction="none").mean(dim=1)

        # Gradient accumulation
        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step(loss=loss)  # Perform parameter update
            optimizer.zero_grad()  # Reset gradients
            scheduler.step()
        else:
            optimizer.virtual_step(loss=loss)  # Accumulate gradients


        # Save the model if this is the best loss so far
        avg_loss = torch.mean(loss).item()
        total_loss += avg_loss

        # Check if the current loss is the best
        if avg_loss < best_loss:
            best_loss = avg_loss

            # Save model and tokenizer using save_pretrained
            model.save_pretrained(checkpoint_dir)
            tokenizer.save_pretrained(checkpoint_dir)

            # Save optimizer state and best_loss separately
            torch.save({
                'optimizer_state_dict': optimizer.state_dict(),
                'best_loss': best_loss,
            }, optimizer_checkpoint_path)

            print(f"New best model saved with loss: {best_loss}")

        # # Log progress
        if i % 10 == 0:
            print(f"Epoch: {epoch + 1}, Step: {i}/{len(data_loader)}, Loss: {avg_loss}", flush=True)

    # Calculate the average training loss and perplexity for the epoch
    avg_train_loss = total_loss / len(data_loader)
    train_perplexity = np.exp(avg_train_loss)
    wandb.log({"epoch": epoch + 1, "train_loss": avg_train_loss, "train_perplexity": train_perplexity})

wandb.finish()


wandb: Currently logged in as: ivolinengong. Use `wandb login --relogin` to force relogin


Starting training from the pretrained model


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/users/k/n/kngongiv/miniconda3/envs/pytorch_envs/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]


Training dataset size: 14732
Validation dataset size: 818
Sample keys: dict_keys(['input_ids', 'attention_mask', 'labels'])
Input IDs shape: torch.Size([150])
Attention mask shape: torch.Size([150])
Labels shape: torch.Size([100])

Starting epoch 1/8


/users/k/n/kngongiv/miniconda3/envs/pytorch_envs/lib/python3.10/site-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [ ]:
# import torch
# from transformers import T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from private_transformers import PrivacyEngine
# import torch.nn.functional as F
# # from transformers import AdamW, get_linear_schedule_with_warmup
# # from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, get_linear_schedule_with_warmup
# from torch.optim import AdamW  
# import numpy as np
# import os
# import wandb
# import spacy

# # Load the SpaCy model for sentence tokenization
# spacy_model = spacy.load("en_core_web_sm")


# # def preprocess_function(examples):
# #     # Use spacy to extract first 3 sentences as a simplification
# #     def get_first_3_sentences(text):
# #         doc = spacy_model(text)
# #         sentences = [sent.text.strip() for sent in doc.sents][:3]
# #         return " ".join(sentences)
    
# #     # Modify here to use 'dialogue' field from SAMSum dataset
# #     inputs = ["summarize: " + get_first_3_sentences(doc) for doc in examples["dialogue"]]
# #     model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

# #     # 'summary' field is used as labels in SAMSum
# #     with tokenizer.as_target_tokenizer():
# #         labels = tokenizer(examples["summary"], max_length=max_output_length, padding="max_length", truncation=True)

# #     model_inputs["input_ids"] = torch.tensor(model_inputs["input_ids"])
# #     model_inputs["attention_mask"] = torch.tensor(model_inputs["attention_mask"])
# #     model_inputs["labels"] = torch.tensor(labels["input_ids"])
# #     return model_inputs

# def preprocess_function(examples):
#     inputs = ["summarize: " + doc for doc in examples["dialogue"]]
#     model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

#     # 'summary' field is used as labels in SAMSum
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples["summary"], max_length=max_output_length, padding="max_length", truncation=True)

#     model_inputs["input_ids"] = torch.tensor(model_inputs["input_ids"])
#     model_inputs["attention_mask"] = torch.tensor(model_inputs["attention_mask"])
#     model_inputs["labels"] = torch.tensor(labels["input_ids"])
#     return model_inputs

# def find_latest_checkpoint(checkpoint_dir):
#     checkpoint_subdirs = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))]
#     if not checkpoint_subdirs:
#         return None
#     latest_checkpoint = max(checkpoint_subdirs, key=lambda d: int(d.split('-')[-1]))
#     return os.path.join(checkpoint_dir, latest_checkpoint)

# def compute_average_length(dataset):
#     lengths = dataset["lengths"]  # Assuming 'lengths' are stored in the dataset
#     average_length = np.mean(lengths)
#     print(f"Average token length: {average_length}")

# wandb.init(project="Summarize_DP_PubMed", entity="ivolinengong")


# # Check if CUDA is available and set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Set seeds for reproducibility
# torch.manual_seed(42)

# # Load the tokenizer
# max_input_length = 150 #1024
# max_output_length = 100 #256 #250
# batch_size = 5
# epochs = 8
# gradient_accumulation_steps = 10

# # Load pubmed dataset
# # Load SAMSum dataset
# train_dataset = load_dataset("samsum", split="train")
# val_dataset = load_dataset("samsum", split="validation")

# # tokenized_datasets = dataset.map(preprocess_function, batched=True)
# # train_dataset = tokenized_datasets["train")

# # Checkpoint directories
# # checkpoint_dir = "./results/dp_results/test/run"
# checkpoint_dir = f"./results/dp_results/{wandb.run.name}"
# # checkpoint_dir = f"./results/dp_results/test/rich-serenity-9"
# os.makedirs(checkpoint_dir, exist_ok=True)

# model_tokenizer_path = checkpoint_dir
# optimizer_checkpoint_path = os.path.join(checkpoint_dir, "optimizer_and_loss.pth")

# # Load model and tokenizer, and optimizer state if available
# # model_checkpoint = find_latest_checkpoint(checkpoint_dir)
# # if model_checkpoint:
# #     print(f"Resuming from checkpoint: {model_checkpoint}")
# #     # model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)
# #     # tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# #     model = T5ForConditionalGeneration.from_pretrained("t5-large").to(device)
# #     tokenizer = T5Tokenizer.from_pretrained("t5-large", model_max_length=max_input_length)

# #     if os.path.isfile(optimizer_checkpoint_path):
# #         print("Loading optimizer state and best loss from checkpoint")
# #         checkpoint = torch.load(optimizer_checkpoint_path, map_location=device)
# #         optimizer = AdamW(model.parameters(), lr=5e-5)
# #         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# #         best_loss = checkpoint['best_loss']
# model_checkpoint = find_latest_checkpoint(checkpoint_dir)
# if model_checkpoint:
#     print(f"Resuming from checkpoint: {model_checkpoint}")

#     # Load model and tokenizer from the checkpoint
#     model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
#     tokenizer = T5Tokenizer.from_pretrained(model_checkpoint, model_max_length=max_input_length)

#     # Load optimizer state
#     if os.path.isfile(optimizer_checkpoint_path):
#         print("Loading optimizer state and best loss from checkpoint")
#         checkpoint = torch.load(optimizer_checkpoint_path, map_location=device)
#         optimizer = AdamW(model.parameters(), lr=5e-5)
#         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#         best_loss = checkpoint['best_loss']
# else:
#     print("Starting training from the pretrained model")
#     model = T5ForConditionalGeneration.from_pretrained("t5-large").to(device)
#     tokenizer = T5Tokenizer.from_pretrained("t5-large", model_max_length=max_input_length)

#     # model = AutoModelForSeq2SeqLM.from_pretrained("t5-large").to(device)
#     # tokenizer = AutoTokenizer.from_pretrained("t5-large", model_max_length=max_input_length)
#     optimizer = AdamW(model.parameters(), lr=5e-5)
#     best_loss = float('inf')

# tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
# tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)
# tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# tokenized_val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


# print("\nTraining dataset size:", len(tokenized_train_dataset))
# print("Validation dataset size:", len(tokenized_val_dataset))


# # # Inspect a sample
# sample = tokenized_train_dataset[0]
# print(f"Sample keys: {sample.keys()}")
# print(f"Input IDs shape: {sample['input_ids'].shape}")
# print(f"Attention mask shape: {sample['attention_mask'].shape}")
# print(f"Labels shape: {sample['labels'].shape}")
# # Calculate and print average token length
# # compute_average_length(tokenized_train_dataset)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Attach the privacy engine to the optimizer
# privacy_engine = PrivacyEngine(
#     model,
#     batch_size=batch_size,
#     sample_size=len(tokenized_train_dataset),
#     epochs=epochs,
#     max_grad_norm=1.0,
#     target_epsilon=3,
#     noise_multiplier=0.1,
#     # clipping_mode="ghost",
# )
# privacy_engine.attach(optimizer)

# # Learning rate scheduler
# total_steps = len(tokenized_train_dataset) // batch_size * epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# # Data loader
# data_loader = DataLoader(tokenized_train_dataset, batch_size=batch_size, shuffle=True)
# val_data_loader = DataLoader(tokenized_val_dataset, batch_size=batch_size)


# # Training loop
# model.train()
# for epoch in range(epochs):
#     total_loss = 0
#     print(f"\nStarting epoch {epoch + 1}/{epochs}", flush=True)
#     for i, batch in enumerate(data_loader):
#         # Move batch to device
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         # Forward pass
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         logits = outputs.logits

#         # Compute per-example loss
#         shift_logits = logits[..., :-1, :].contiguous()
#         shift_labels = labels[..., 1:].contiguous()
#         loss = F.cross_entropy(shift_logits.permute(0, 2, 1), shift_labels, reduction="none").mean(dim=1)

#         # Gradient accumulation
#         if (i + 1) % gradient_accumulation_steps == 0:
#             optimizer.step(loss=loss)  # Perform parameter update
#             optimizer.zero_grad()  # Reset gradients
#             scheduler.step()
#         else:
#             optimizer.virtual_step(loss=loss)  # Accumulate gradients


#         # Save the model if this is the best loss so far
#         avg_loss = torch.mean(loss).item()
#         total_loss += avg_loss

#         # Check if the current loss is the best
#         if avg_loss < best_loss:
#             best_loss = avg_loss

#             # Save model and tokenizer using save_pretrained
#             model.save_pretrained(checkpoint_dir)
#             tokenizer.save_pretrained(checkpoint_dir)

#             # Save optimizer state and best_loss separately
#             torch.save({
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'best_loss': best_loss,
#             }, optimizer_checkpoint_path)

#             print(f"New best model saved with loss: {best_loss}")

#         # # Log progress
#         if i % 10 == 0:
#             print(f"Epoch: {epoch + 1}, Step: {i}/{len(data_loader)}, Loss: {avg_loss}", flush=True)

#     # Calculate the average training loss and perplexity for the epoch
#     avg_train_loss = total_loss / len(data_loader)
#     train_perplexity = np.exp(avg_train_loss)
#     wandb.log({"epoch": epoch + 1, "train_loss": avg_train_loss, "train_perplexity": train_perplexity})

# wandb.finish()